# Quantum Simulation Project (SProject)

Notebook này sử dụng cấu trúc modular đã được tách riêng để thực hiện các tính toán quantum simulation.

## Các module chính:
- **quantum_setup**: Setup molecular problem và quantum operators
- **vqe_optimization**: VQE optimization algorithms
- **vqc_optimization**: VQC/VQA quantum compilation
- **time_evolution**: Time evolution calculations
- **ansatz**: Ansatz definitions
- **visualization**: Plotting functions
- **utils**: Utility functions
- **optimization**: Common optimization utilities

In [1]:
# Import các module chính
from quantum_setup import (
    setup_molecular_problem, 
    setup_qubit_operators, 
    setup_ansatz, 
    get_exact_ground_state
)

from vqe_optimization import (
    run_vqe, 
    get_vqe_ground_state, 
    run_vqe_with_multiple_optimizers,
    VQETracker
)

from vqc_optimization import (
    run_vqa_time_evolution,
    evolve_states_with_vqa_unitaries,
    calculate_dipole_moments_from_states,
    VQCTracker,
    train_vqa_for_time
)

from time_evolution import (
    compute_target_unitaries, 
    create_time_evolution_function
)

from ansatz import create_pauli_labels_from_operators

from visualization import (
    plot_optimization_convergence, 
    plot_energy_convergence, 
    plot_electric_field,
    setup_matplotlib_style
)

from optimization import (
    OptimizationTracker,
    unitary_fidelity,
    save_cost_history
)

from utils import save_results, create_timestamp, print_system_info
import config
import numpy as np

print("✅ Tất cả các module đã được import thành công!")

✅ Tất cả các module đã được import thành công!


In [ ]:
# Setup cấu hình và molecular problem
print("Setting up molecular problem...")

# Setup matplotlib style
setup_matplotlib_style()

# Print system information
print_system_info()

# Setup molecular problem
problem, hamiltonian, dipole = setup_molecular_problem()
print(f"✅ Molecular problem setup complete")
print(f"   - Molecule: {config.MOLECULE_GEOMETRY}")
print(f"   - Basis: {config.BASIS_SET}")
print(f"   - Bond distance: {config.BOND_DISTANCE} Å")

# Setup qubit operators
qubit_ops = setup_qubit_operators(hamiltonian, dipole)
print(f"✅ Qubit operators setup complete")
print(f"   - Number of qubits: {qubit_ops['num_qubits']}")
print(f"   - Hamiltonian operator shape: {qubit_ops['hamiltonian_qubit'].num_terms} terms")

Setting up molecular problem...
System Information:
Python version: 3.10.12 (main, May 27 2025, 17:12:29) [GCC 11.4.0]
Platform: Linux-5.15.153.1-microsoft-standard-WSL2-x86_64-with-glibc2.35
Processor: x86_64
Machine: x86_64
Qiskit version: 1.0.2
PennyLane version: 0.38.0
✅ Molecular problem setup complete
   - Molecule: H 0 0 0; H 0 0 2.0
   - Basis: sto3g
✅ Qubit operators setup complete


KeyError: 'hamiltonian_qubit'

In [ ]:
## 1. VQE Optimization

Thực hiện VQE để tìm ground state energy của phân tử.

In [8]:
# Chạy VQE với optimizer đơn
print("Chạy VQE optimization...")

vqe_result = run_vqe(
    qubit_ops['hamiltonian_qubit'], 
    qubit_ops['num_qubits']
)

print(f"✅ VQE hoàn thành")
print(f"   - Ground state energy: {vqe_result['optimal_value']:.6f} Ha")
print(f"   - Optimal parameters shape: {vqe_result['optimal_parameters'].shape}")
print(f"   - Optimizer convergence: {vqe_result['optimizer_result'].success}")

# Lấy exact ground state để so sánh
exact_energy = get_exact_ground_state(qubit_ops['hamiltonian_qubit'])
energy_error = abs(vqe_result['optimal_value'] - exact_energy)
print(f"   - Exact energy: {exact_energy:.6f} Ha")
print(f"   - Energy error: {energy_error:.8f} Ha")

Chạy VQE optimization...


KeyError: 'hamiltonian_qubit'

In [ ]:
# So sánh nhiều optimizer
print("So sánh hiệu suất của các optimizer khác nhau...")

vqe_comparison = run_vqe_with_multiple_optimizers(
    qubit_ops['hamiltonian_qubit'], 
    qubit_ops['num_qubits'],
    optimizers=['SLSQP', 'COBYLA', 'L_BFGS_B']
)

print("\nKết quả so sánh optimizer:")
for optimizer, result in vqe_comparison.items():
    print(f"  {optimizer}:")
    print(f"    - Energy: {result['optimal_value']:.6f} Ha")
    print(f"    - Error: {abs(result['optimal_value'] - exact_energy):.8f} Ha")
    print(f"    - Convergence: {result['optimizer_result'].success}")
    print(f"    - Function evaluations: {result['optimizer_result'].nfev}")

In [ ]:
## 2. Time Evolution Setup

Tính toán time evolution để chuẩn bị cho VQA quantum compilation.

In [ ]:
# Tính toán target unitaries cho time evolution
print("Tính toán target unitaries...")

# Tạo time points
times = np.linspace(0, config.MAX_TIME, config.NUM_TIME_POINTS)
print(f"Time points: {len(times)} từ 0 đến {config.MAX_TIME}")

# Compute target unitaries
target_unitaries = compute_target_unitaries(
    qubit_ops['hamiltonian_static'], 
    times
)

print(f"✅ Target unitaries computed")
print(f"   - Shape của mỗi unitary: {target_unitaries[0].shape}")
print(f"   - Số unitary matrices: {len(target_unitaries)}")

# Kiểm tra tính unitary
is_unitary = np.allclose(
    target_unitaries[0] @ target_unitaries[0].conj().T, 
    np.eye(target_unitaries[0].shape[0]), 
    atol=1e-10
)
print(f"   - Kiểm tra tính unitary: {is_unitary}")

In [ ]:
## 3. VQA Quantum Compilation

Sử dụng VQA để học time evolution unitaries.

In [ ]:
# Setup ansatz cho VQA
print("Setup ansatz cho VQA...")

# Tạo Pauli labels từ Hamiltonian
pauli_labels = create_pauli_labels_from_operators([qubit_ops['hamiltonian_qubit']])
print(f"✅ Pauli labels created: {len(pauli_labels)} terms")

# Chạy VQA time evolution
print("Bắt đầu VQA time evolution...")
vqa_results = run_vqa_time_evolution(
    target_unitaries, 
    times,
    qubit_ops['num_qubits'], 
    pauli_labels,
    num_layers=config.VQA_NUM_LAYERS,
    steps=config.VQA_STEPS,
    learning_rate=config.VQA_LEARNING_RATE
)

print(f"✅ VQA time evolution hoàn thành")
print(f"   - Số time points: {len(vqa_results['optimized_unitaries'])}")
print(f"   - Final parameters shape: {vqa_results['final_thetas'].shape}")

In [ ]:
# Phân tích fidelity và cost
print("Phân tích fidelity...")

fidelities = []
for i, t in enumerate(times):
    if t in vqa_results['optimized_unitaries']:
        target_unitary = target_unitaries[i]
        optimized_unitary = vqa_results['optimized_unitaries'][t]
        
        fidelity = unitary_fidelity(target_unitary, optimized_unitary)
        fidelities.append(fidelity)

fidelities = np.array(fidelities)
print(f"✅ Fidelity analysis:")
print(f"   - Mean fidelity: {np.mean(fidelities):.6f}")
print(f"   - Min fidelity: {np.min(fidelities):.6f}")
print(f"   - Max fidelity: {np.max(fidelities):.6f}")
print(f"   - Std fidelity: {np.std(fidelities):.6f}")

# Lưu cost history cho time point cuối
final_time = times[-1]
if final_time in vqa_results['cost_histories']:
    final_cost_history = vqa_results['cost_histories'][final_time]
    save_cost_history(final_cost_history, "sproject_final_cost_history.txt")
    print(f"✅ Cost history saved for t = {final_time}")

In [ ]:
## 4. State Evolution và Dipole Moments

Tính toán state evolution và dipole moments từ VQA results.

In [ ]:
# Tính toán evolved states
print("Tính toán evolved states...")

# Initial state (ground state từ VQE)
initial_state = vqe_result['eigenstate']
print(f"Initial state shape: {initial_state.shape}")

# Evolve states với VQA unitaries
evolved_states = evolve_states_with_vqa_unitaries(
    vqa_results['optimized_unitaries'], 
    initial_state, 
    times
)

print(f"✅ States evolved:")
print(f"   - Số states: {len(evolved_states)}")

# Tính dipole moments
print("Tính toán dipole moments...")
times_dipole, dipole_moments = calculate_dipole_moments_from_states(
    evolved_states, 
    qubit_ops['dipole_qubit'], 
    times
)

print(f"✅ Dipole moments calculated:")
print(f"   - Time points: {len(times_dipole)}")
print(f"   - Dipole range: [{np.min(dipole_moments):.6f}, {np.max(dipole_moments):.6f}]")
print(f"   - Mean dipole: {np.mean(dipole_moments):.6f}")

In [ ]:
## 5. Visualization

Tạo các biểu đồ để phân tích kết quả.

In [ ]:
# Plot VQE convergence cho các optimizer
print("Tạo biểu đồ VQE convergence...")

fig_vqe = plot_energy_convergence(vqe_comparison, exact_energy)
fig_vqe.suptitle('VQE Optimizer Comparison - SProject', fontsize=14)
fig_vqe.tight_layout()
fig_vqe.show()

print("✅ VQE convergence plot created")

In [ ]:
# Plot VQA cost history cho time point cuối
if final_time in vqa_results['cost_histories']:
    print("Tạo biểu đồ VQA convergence...")
    
    fig_vqa = plot_optimization_convergence(
        [vqa_results['cost_histories'][final_time]], 
        [f"VQA t={final_time:.3f}"]
    )
    fig_vqa.suptitle(f'VQA Convergence at t={final_time:.3f} - SProject', fontsize=14)
    fig_vqa.show()

# Plot dipole moments
print("Tạo biểu đồ dipole moments...")
import matplotlib.pyplot as plt

fig_dipole, ax = plt.subplots(figsize=(10, 6))
ax.plot(times_dipole, dipole_moments, 'b-', linewidth=2, label='VQA Dipole Moment')
ax.set_xlabel('Time (a.u.)')
ax.set_ylabel('Dipole Moment (a.u.)')
ax.set_title('Time Evolution of Dipole Moment - SProject')
ax.grid(True, alpha=0.3)
ax.legend()
fig_dipole.tight_layout()
fig_dipole.show()

print("✅ Tất cả các biểu đồ đã được tạo")

# Tóm tắt kết quả
print("\n" + "="*60)
print("TÓM TẮT KẾT QUẢ SPROJECT")
print("="*60)
print(f"VQE Ground State Energy: {vqe_result['optimal_value']:.6f} Ha")
print(f"Energy Error: {abs(vqe_result['optimal_value'] - exact_energy):.8f} Ha")
print(f"VQA Mean Fidelity: {np.mean(fidelities):.6f}")
print(f"Dipole Moment Range: [{np.min(dipole_moments):.6f}, {np.max(dipole_moments):.6f}]")
print(f"Timestamp: {create_timestamp()}")
print("="*60)